# Chopper Cascades

## Overview

The [chopper_cascade module](https://scipp.github.io/scippneutron/generated/modules/scippneutron.tof.chopper_cascade.html) provides utilities for computing time and wavelength bounds of a neutron pulse (or sub-pulses) propagating through a chopper cascade.
This is useful for designing chopper systems, as well as predicting the data recorded when using techniques such as wavelength-frame multiplication (WFM).

It is currently under development and not fully functional.

## Example: WFM chopper cascade

As an example, consider the WFM chopper cascade from [the tof package documentation](https://tof.readthedocs.io/en/stable/short-example.html).

In [ ]:
from scippneutron.tof import chopper_cascade
import scipp as sc
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 300

### Defining choppers

Choppers are defined by their position along the beam path,
and the opening and closing times of their cutouts.

In [ ]:
params = {'wfm1': {'open': [-0.000396, 0.001286, 0.005786, 0.008039, 0.010133, 0.012080, 0.013889, 0.015571],
                   'close': [0.000654, 0.002464, 0.006222, 0.008646, 0.010899, 0.012993, 0.014939, 0.016750],
                   'distance': 6.6},
          'wfm2': {'open': [0.000654, 0.002451, 0.006222, 0.008645, 0.010898, 0.012993, 0.014940, 0.016737],
                   'close': [0.001567, 0.003641, 0.006658, 0.009252, 0.011664, 0.013759, 0.015853, 0.017927],
                   'distance': 7.1},
          'foc1': {'open': [-0.000139, 0.002460, 0.006796, 0.010020, 0.012733, 0.015263, 0.017718, 0.020317],
                   'close': [0.000640, 0.003671, 0.007817, 0.011171, 0.013814, 0.016146, 0.018497, 0.021528],
                   'distance': 8.8},
          'foc2': {'open': [-0.000306, 0.010939, 0.016495, 0.021733, 0.026416, 0.030880, 0.035409],
                   'close': [0.002582, 0.014570, 0.020072, 0.024730, 0.029082, 0.033316, 0.038297],
                   'distance': 15.9}
         }

choppers = sc.DataGroup(
    {name: chopper_cascade.Chopper(
        distance=sc.scalar(param['distance'], unit='m'),
        time_open=sc.array(dims=('cutout',), values=param['open'], unit='s'),
        time_close=sc.array(dims=('cutout',), values=param['close'], unit='s')
    ) for name, param in params.items()}
)

We can now initialize a frame-sequence and apply the chopper cascade to it:

In [ ]:
frames = chopper_cascade.FrameSequence.from_source_pulse(
    time_min=sc.scalar(0.0, unit='ms'),
    time_max=sc.scalar(4.0, unit='ms'),  # ESS pulse is 3 ms, but it has a tail
    wavelength_min=sc.scalar(0.0, unit='angstrom'),
    wavelength_max=sc.scalar(10.0, unit='angstrom'),
)
frames = frames.chop(choppers.values())
at_sample = frames.propagate_to(sc.scalar(26.0, unit='m'))

at_sample.draw()

We can also draw a chopper acceptance diagram, which is essentially the same as above, but propagated back to the source pulse distance:

In [ ]:
frames.acceptance_diagram()

## Frame unwrapping

For unwrapping frames, we need the bounds of the entire frame, to determine times at which to cut.
Since $L_2$ can be different for every detector, this cutting time is different for every detector.
We can compute the frame bounds at a common distance, e.g., the sample, and propagate the bounds to the detectors:

In [ ]:
bounds = at_sample[-1].bounds()
chopper_cascade.propagate_times(
    time=bounds['time'],
    wavelength=bounds['wavelength'],
    distance=sc.linspace('L2', 1.0, 2.0, 100, unit='m'),
)

For WFM, we need to compute subframe time cutting points.
Again, $L_2$ can be different for every detector, so we need to compute the cutting points for every detector:

In [ ]:
bounds = at_sample[-1].subbounds()
bounds

In [ ]:
chopper_cascade.propagate_times(
    time=bounds['time'],
    wavelength=bounds['wavelength'],
    distance=sc.linspace('L2', 1.0, 2.0, 100, unit='m'),
)

Note that subframes may in principle overlap, and this may depend on the detector.
We therefore call `subframe_bounds` at a common location and propagate the result, otherwise we would get a different number of subframes for different detectors.